In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_score, recall_score, classification_report, f1_score)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
!pip install dagshub
!pip install mlflow
import dagshub
import mlflow

dagshub.init(repo_owner='kechik21',repo_name='ML_HW2',mlflow=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 6.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 57.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.5 MB/s eta 0:00:00
   

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=90c536d1-5dfc-4c22-acfa-15f6a3a6e18d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b039945e95784933307752b6e6472bd60085c6efe3e7b380efdc97c757bcf959




Output()

Accessing as kechik21

Initialized MLflow to track repo "kechik21/ML_HW2"

Repository kechik21/ML_HW2 initialized!

In [3]:
# Training data
train_tr = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')

train = train_tr.merge(train_id, on='TransactionID', how='left')
print(train_tr.shape)
print(train_id.shape)

(590540, 394)
(144233, 41)


In [4]:
print("Train Data Columns:")
print(train.columns)

Train Data Columns:
Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)


In [5]:
X = train.drop(['TransactionID', 'isFraud'], axis=1)
y = train['isFraud']

In [6]:
# Differentiate categorical and numerical
cat_coluka = ['userId','P_emaildomain','R_emaildomain','DeviceType','DeviceInfo','ProductCD','addr1', 'addr2',]
cat_cards = ['card' + str(i) for i in range(1, 7)]
cat_ms = ['M' + str(i) for i in range(1, 10)]
cat_ids = ['id_' + str(i) for i in range(12, 39)]
cat_cols = cat_coluka + cat_cards + cat_ms + cat_ids

print("Categorical Columns:")
print(cat_cols)
print("\nCard Columns:")
print(cat_cards)
print("\nM Columns:")
print(cat_ms)
print("\nID Columns:")
print(cat_ids)
print("\nAll Categorical Features:")
print(cat_cols)
print("\nCategorical Columns Amount:")
print(len(cat_cols))


num_cols = [col for col in X.columns if col not in cat_cols and col != 'isFraud']

print("\nNumerical Columns Amount:")
print(len(num_cols))

Categorical Columns:
['userId', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo', 'ProductCD', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

Card Columns:
['card1', 'card2', 'card3', 'card4', 'card5', 'card6']

M Columns:
['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']

ID Columns:
['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

All Categorical Features:
['userId', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo', 'ProductCD', 'addr1', 'addr2',

Filling missing values for numeric and categorical

In [7]:
class FillNa(BaseEstimator, TransformerMixin):
    def __init__(self, num_cols, cat_cols):
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.values = {}

    def fit(self, X, y=None):
        for col in self.num_cols:
            if col in X.columns:
                val = X[col].median()
                X[col] = X[col].fillna(val)
                self.values[col] = val

        for c in self.cat_cols:
            if c in X.columns:  
                if X[c].dtype in ['int64', 'float64']:
                    X[c] = X[c].astype(str)
                X[c] = X[c].fillna('NotAv')
        return self

    def transform(self, X):
        for key, value in self.values.items():
            if key in X.columns:
                X[key] = X[key].fillna(value)

        for c in self.cat_cols:
            if c in X.columns:  
                if X[c].dtype in ['int64', 'float64']:
                    X[c] = X[c].astype(str)
                X[c] = X[c].fillna('NotAv')

        return X

In [8]:
class MissingValueDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=90):
        self.columns_to_drop = None
        self.threshold = threshold

    def fit(self, X, y=None):
        missing_cols = [col for col in X.columns if X[col].isna().any()]
        high_missing_cols = [
            col for col in missing_cols 
            if (X[col].isna().mean() * 100) > self.threshold
        ]
        
        self.columns_to_drop = high_missing_cols
        print(f'Identified {len(high_missing_cols)} columns to drop')
        return self

    def transform(self, X):
        if self.columns_to_drop is None:
            raise ValueError('Needs Fitting !')
        
        return X.drop(columns=self.columns_to_drop)

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import WOEEncoder
import pandas as pd

class CatEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=2):
        self.threshold = threshold
        self.binary_cols = []
        self.multi_cols = []
        self.encoder = None

    def fit(self, X, y):
        X = X.copy()
        
        for col in X.columns:
            if X[col].dtype == 'object':
                non_rep = X[col].nunique()
                if non_rep <= self.threshold:
                    self.binary_cols.append(col)
                else:
                    self.multi_cols.append(col)

        self.encoder = WOEEncoder(cols=self.multi_cols)
        self.encoder.fit(X[self.multi_cols], y)
        
        return self

    def transform(self, X):
        X = X.copy()
        if self.multi_cols:
            X_woe = self.encoder.transform(X[self.multi_cols])
            X[self.multi_cols] = X_woe

        if self.binary_cols:
            X = pd.get_dummies(X, columns=self.binary_cols, drop_first=True, dtype=int)
        
        return X

In [10]:
class HighCorrelationDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.85):
        self.threshold = threshold
        self.high_corr_cols_ = None  
        
    def fit(self, X, y=None):
       
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame")
        corr_matrix = X.corr().abs()
        upper_triangle = np.triu(corr_matrix, k=1)
        highly_correlated = set()
        for col in corr_matrix.columns:
            correlated_with = corr_matrix.index[upper_triangle[:, corr_matrix.columns.get_loc(col)] > self.threshold]
            highly_correlated.update(correlated_with)
            
        self.high_corr_cols_ = list(highly_correlated)
        print(f"Identified {len(self.high_corr_cols_)} highly correlated features to remove")
        
        return self
        
    def transform(self, X):
        if self.high_corr_cols_ is None:
            raise RuntimeError("Need Fitting ! ")
            
        return X.drop(columns=self.high_corr_cols_)

In [11]:
class SimpleLogisticFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, target_percentage=0.3, C=0.1, random_state=42):
        self.target_percentage = target_percentage
        self.C = C
        self.random_state = random_state
        self.kept_features_ = None
        self.model = None
        
    def fit(self, X, y):
        print("\nStarting feature selection...")
        print(f"Input shape: {X.shape}")
        
        self.model = LogisticRegression(
            C=self.C,
            penalty='l1',
            solver='saga',  
            random_state=self.random_state,
            max_iter=1000,  
            tol=1e-4,  
            class_weight='balanced',
            verbose=1  
        )
        
        print("Fitting feature selector model...")
        self.model.fit(X, y)
        print("Feature selector model fitted successfully")
        
        importances = np.abs(self.model.coef_[0])
        num_to_keep = max(20, int(X.shape[1] * self.target_percentage))
        
        indices = np.argsort(importances)[::-1][:num_to_keep]
        mask = np.zeros(X.shape[1], dtype=bool)
        mask[indices] = True
        
        if hasattr(X, 'columns'):
            self.kept_features_ = X.columns[mask].tolist()
        else:
            self.kept_features_ = list(range(X.shape[1]))[mask]
            
        print(f"Feature selection complete. Selected {len(self.kept_features_)} features.")
        return self
        
    def transform(self, X):
        if self.kept_features_ is None:
            raise RuntimeError("Fit the selector first!")
        
        if isinstance(X, pd.DataFrame):
            return X[self.kept_features_]
        return X[:, self.kept_features_]


In [12]:
class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        
    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self
        
    def transform(self, X):
        return pd.DataFrame(
            self.scaler.transform(X),
            columns=X.columns,
            index=X.index
        )

In [13]:
print('The Classes I created are working so far')

The Classes I created are working so far


In [14]:
kva = train.drop(['TransactionID', 'isFraud'], axis=1)
target = train['isFraud']

X_train, X_val, y_train, y_val = train_test_split(kva,target,test_size=0.3,stratify=target,random_state=42)

In [ ]:
mlflow.set_tracking_uri('https://dagshub.com/kechik21/ML_HW2.mlflow')
mlflow.set_experiment("Fraud_Detection_LogisticRegression")


class_weights = {
    0: 1.0,
    1: (len(y_train) - sum(y_train)) / sum(y_train)
}


fraud_pipeline = Pipeline([
    ('missing_value_handler', MissingValueDropper(threshold=90)),
    ('data_imputer', FillNa(num_cols=num_cols, cat_cols=cat_cols)),
    
    ('categorical_processor', CatEncoder(threshold=2)),
    ('correlation_filter', HighCorrelationDropper(threshold=0.85)),
    
    ('feature_selector', SimpleLogisticFeatureSelector(
        target_percentage=0.3,
        C=0.1,
        random_state=42
    )),
    
    ('scaler', FeatureScaler()),
    
    ('fraud_classifier', LogisticRegression(
        C=0.1,
        penalty='l2',
        solver='saga',
        max_iter=2000,
        class_weight=class_weights,
        n_jobs=-1,
        random_state=42,
        verbose=1  # Show training progress
    ))
])

with mlflow.start_run(run_name="Production_LR_Fraud_Model"):
    mlflow.log_params({
        "pipeline_steps": [name for name, _ in fraud_pipeline.steps],
        "class_imbalance_ratio": class_weights[1],
        "feature_selection": "Logistic_L1",
        "regularization": "l2",
        "C": 0.1,
        "feature_selection_percentage": 0.3
    })
    
    print("\nStarting pipeline execution...")
    fraud_pipeline.fit(X_train, y_train)
    print("\nTraining completed successfully")
    
    fraud_probabilities = fraud_pipeline.predict_proba(X_val)[:, 1]
    fraud_predictions = fraud_pipeline.predict(X_val)
    
    # Calculate metrics
    pipeline_metrics = {
        "validation_auc": roc_auc_score(y_val, fraud_probabilities),
        "validation_ap": average_precision_score(y_val, fraud_probabilities),
        "validation_f1": f1_score(y_val, fraud_predictions),
        "fraud_precision": precision_score(y_val, fraud_predictions),
        "fraud_recall": recall_score(y_val, fraud_predictions)
    }
    
    mlflow.log_metrics(pipeline_metrics)
    mlflow.log_dict(
        classification_report(y_val, fraud_predictions, output_dict=True),
        "classification_report.json"
    )
    
    
    coefficients = lr_model.coef_[0]
    importance = np.abs(coefficients)
    indices = np.argsort(importance)[::-1][:20]  # Top 20 features
    
    
    mlflow.sklearn.log_model(
        fraud_pipeline,
        "lr_fraud_detection_model",
        registered_model_name="Production_LR_Fraud_Pipeline"
    )
    
    print(f"\nLR Pipeline deployed | Validation F1: {pipeline_metrics['validation_f1']:.4f}")
    print("Run completed successfully!")


Starting pipeline execution...
Identified 12 columns to drop
Identified 238 highly correlated features to remove

Starting feature selection...
Input shape: (413378, 182)
Fitting feature selector model...
Epoch 1, change: 1.00000000
Epoch 2, change: 0.15517152
Epoch 3, change: 0.03597195
Epoch 4, change: 0.02200391
Epoch 5, change: 0.03495926
Epoch 6, change: 0.04767360
Epoch 7, change: 0.06212493
Epoch 8, change: 0.03652594
Epoch 9, change: 0.02032034
Epoch 10, change: 0.03769116
Epoch 11, change: 0.04377773
Epoch 12, change: 0.02018432
Epoch 13, change: 0.05048555
Epoch 14, change: 0.05600155
Epoch 15, change: 0.01907791
Epoch 16, change: 0.03035034
Epoch 17, change: 0.01936142
Epoch 18, change: 0.01896346
Epoch 19, change: 0.01840804
Epoch 20, change: 0.01952278
Epoch 21, change: 0.04393462
Epoch 22, change: 0.06053025
Epoch 23, change: 0.01827738
Epoch 24, change: 0.02028141
Epoch 25, change: 0.02638685
Epoch 26, change: 0.01775595
Epoch 27, change: 0.02267187
Epoch 28, change: 0.